In [6]:
from sqlalchemy import create_engine
import pandas as pd

user = 'jupyter_user'
password = '666-Zelda'
host = 'localhost'
port = '3306'
database = 'sakila'

engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [7]:
engine.connect()

In [11]:
import pandas as pd
from sqlalchemy import text

def rentals_month(engine, month, year):
    query = text("""
        SELECT * 
        FROM rental
        WHERE MONTH(rental_date) = :month
          AND YEAR(rental_date) = :year
    """)  
    df = pd.read_sql_query(query, engine, params={"month": month, "year": year})
    return df

In [12]:
df_rentals = rentals_month(engine, 6, 2005)
print(df_rentals.head())

   rental_id         rental_date  inventory_id  customer_id  \
0       1158 2005-06-14 22:53:33          1632          416   
1       1159 2005-06-14 22:55:13          4395          516   
2       1160 2005-06-14 23:00:34          2795          239   
3       1161 2005-06-14 23:07:08          1690          285   
4       1162 2005-06-14 23:09:38           987          310   

          return_date  staff_id         last_update  
0 2005-06-18 21:37:33         2 2006-02-15 21:30:53  
1 2005-06-17 02:11:13         1 2006-02-15 21:30:53  
2 2005-06-18 01:58:34         2 2006-02-15 21:30:53  
3 2005-06-21 17:12:08         1 2006-02-15 21:30:53  
4 2005-06-23 22:00:38         1 2006-02-15 21:30:53  


In [13]:
def rental_count_month(df, month, year):
    column_name = f"rentals_{month:02d}_{year}"
    
    rental_counts = (
        df.groupby("customer_id")
          .size()
          .reset_index(name=column_name)
    )
    
    return rental_counts

In [14]:
df_rentals = rentals_month(engine, 5, 2005)
df_counts = rental_count_month(df_rentals, 5, 2005)
print(df_counts.head())

   customer_id  rentals_05_2005
0            1                2
1            2                1
2            3                2
3            5                3
4            6                3


In [15]:
def compare_rentals(df1, df2):
    combined = pd.merge(df1, df2, on="customer_id", how="outer").fillna(0)

    col1 = df1.columns[1]
    col2 = df2.columns[1]

    combined[col1] = combined[col1].astype(int)
    combined[col2] = combined[col2].astype(int)

    combined["difference"] = combined[col2] - combined[col1]

    return combined

In [16]:
df_may = rental_count_month(rentals_month(engine, 5, 2005), 5, 2005)
df_june = rental_count_month(rentals_month(engine, 6, 2005), 6, 2005)

comparison = compare_rentals(df_may, df_june)
print(comparison.head())

   customer_id  rentals_05_2005  rentals_06_2005  difference
0            1                2                7           5
1            2                1                1           0
2            3                2                4           2
3            4                0                6           6
4            5                3                5           2
